# 🗺️ ST-Visions Live Demo 1: Introduction and Plain Visualization 

This is a live demonstration of **ST-Visions**, a Python library for interactive and customizable spatio-temporal visualizations using [Bokeh](https://docs.bokeh.org/). The main purpose of this notebook is to visualize the core functions of the library and introduce the bread-and-butter functions that are used for all the use cases

The focus of this notebook will be the following

- Visualizing a Basic Geometry Dataset, showcasing the steps to generate a figure using the library
- Using Filters on the figure, one a time or multiple together
- A Note Towards (Multi)-Polygon Visualization, which will be built further on the 3rd Live Demo (Advanced Use-Cases)


## Library Imports / Initialization

For the purposes of this demo, the following libraries will be imported, this includes the the extra py files of the library that work like modules

- express is a fast track module for plotting that wraps many functions and lets you generate figures out the box
- geom_helper as the name implies helps with the geometry handling, more apparent in more advanced usecases whenever polygon or multi polygon geometry is concerned


The rest of the libraries are the core requirements of the library and they allow geospatial data handling and interactive visualizations

*Note*: the notebook contains tools needed for development as the upgrade of the library is a work in progress, this dev config allows faster development and debugging, allowing in-VSC bokeh calls and auto library reload on changes

In [34]:
import sys, os

# ===== DEV ENVIROMENT CONFIGURATION (TEMPORARY) =====

%load_ext autoreload
%autoreload 2
sys.path.append(os.path.abspath('../../src')) # Ideally ST_Visions will be installed as a module, therefore this is more of a development setting for quick testing.
os.environ["BOKEH_ALLOW_WS_ORIGIN"] = "*" # DEV ENVIROMENT BANDAID FOR VSCODE VISUALIZATIONS, will be edited

# =====================================================

import pandas as pd
import geopandas as gpd
import shapely
import bokeh.models as bokeh_models
from st_visions.visualization.st_visualizer import st_visualizer
import st_visions.express.st_vizexpress as viz_express
import st_visions.visualization.geom_helper as viz_helper

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


If the jupyter notebook is hosted locally, the default parameter of every notebook visualization function call is set as `localhost:localport`. If you are working with a remote server, you can set your server url in the configuration variable below (`NOTEBOOK_URL`) 

In [24]:
NOTEBOOK_URL='http://<NOTEBOOK_IP_ADDRESS>:<NOTEBOOK_PORT>' # add your own server address

##  Simple Geometry Dataset: Visualizing Vessel Movement Data with ST_Visions

This section will show a step by step implementation of a simple problem that the ST Vision library handles, Visualizing a simple Geometry Dataset, meaning a dataset that contains 1-dimensional points

For the purpose of this notebook example, the Piraeus AIS Dataset will be used. To be precise, the December 2017 AIS Snapshot will be loaded 

[1] https://zenodo.org/records/6323416#.YnwPPi8RrAA



### Initialize and populate a VISIONS Instance

The core of the ST_Visions library lies in the instantiation of a VISIONS instance using the st_visualizer class and then populating with a csv. The CSV is parsed as a GeoDataFrame for the geospatial visualization purposes. For the demonstration, we will be limited to 10.000 records from the dataset

In [25]:
st_viz = st_visualizer(limit=5000) # Initialize a VISIONS Instance (ST Visualizer Object)
st_viz.get_data_csv(filepath=r'..\..\data\unipi_ais_dynamic_2017\unipi_ais_dynamic_dec2017.csv', nrows=10000)

### Create a canvas 

Next step is the creation of a bokeh-based canvas for the VISIONS instance. The Canvas creation function is equipped with arguments that equip the function with extra customizability (tools, tile provider, sizing mode, size etc)

Extra keyword arguments pertaining to the WMTSTileSource instance for the map tile provider can be passed through the tile_kwargs dictionary parameter (defaults to an empty dict)

Extra keyword arguments pertaining to the ***figure*** class in bokeh can be passed as a dictionary, more information [here](https://docs.bokeh.org/en/latest/docs/reference/plotting/figure.html#bokeh.plotting.figure)

In [26]:
st_viz.create_canvas(title=f'Prototype Plot', tile_provider="CARTODBPOSITRON", sizing_mode='scale_width', height=540, tools="pan, box_zoom, lasso_select, wheel_zoom, hover, save, reset")

### Add Markers 

using the add_marker function, we can add markers to the points of the records present in the csv we loaded. Vessel Positions are mapped as blue circles with customizable size, marker type, trasparency, legend label etc

In [27]:
circ = st_viz.add_marker(marker='circle', size=10, color='royalblue', alpha=0.7, fill_alpha=0.5, muted_alpha=0, legend_label=f'Vessel GPS Locations')

### Configure Tooltips, Legend and Selection

In [28]:
tooltips = [('Vessel ID','@vessel_id'), ('Timestamp','@t'), ('Speed (knots)','@speed'),
            ('Course over Ground (degrees)','@course'), ('Heading (degrees)','@heading'), ('Coordinates','(@lon, @lat)')]

st_viz.add_hover_tooltips(tooltips)
st_viz.add_lasso_select()

st_viz.figure.legend.location = "top_left"
st_viz.figure.legend.click_policy = "mute"
st_viz.figure.toolbar.active_scroll = st_viz.figure.select_one(bokeh_models.WheelZoomTool)

### Display in Notebook

Renders the interactive plot directly within the Jupyter notebook with responsive sizing.

In [29]:
st_viz.show_figures(notebook=True, height=300, width=300, sizing_mode='stretch_both')

Loading BokehJS ...

## Filtering Vessel Movement Data

The ST_Visions library provides intuitive and flexible filtering capabilities to subset spatial-temporal data for focused analysis. Filters can be applied on numerical, temporal, or categorical columns before visualizing or processing.

To showcase the Filters, the **express** module will be used which fast-tracks the plotting 

### Temporal Filtering

Easily filter records based on timestamps present in the dataset's timestamp column 

In [30]:
st_viz = st_visualizer(limit=1000) # Initialize a VISIONS Instance (ST Visualizer Object)
st_viz.get_data_csv(filepath=r'..\..\data\unipi_ais_dynamic_2017\unipi_ais_dynamic_dec2017.csv', nrows=10000)

tooltips = [('Vessel ID','@vessel_id'), ('Timestamp','@t'), ('Speed (knots)','@speed'),
            ('Course over Ground (degrees)','@course'), ('Heading (degrees)','@heading'), ('Coordinates','(@lon, @lat)')]

viz_express.plot_points_on_map(st_viz, tile_provider='CARTODBPOSITRON', tools=['lasso_select'], tooltips=tooltips)

In [31]:
st_viz.add_temporal_filter(temporal_name='t', temporal_unit='ms', step_ms=180000, title='Temporal Horizon')
st_viz.show_figures(notebook=True)

Loading BokehJS ...

### Numerical Filters

Filter the datapoints based on a numerical column, has 2 different ways to filter

**Range-based:** You can filter values that fall within a specific interval.

In [33]:
st_viz = st_visualizer(limit=1000) # Initialize a VISIONS Instance (ST Visualizer Object)
st_viz.get_data_csv(filepath=r'..\..\data\unipi_ais_dynamic_2017\unipi_ais_dynamic_dec2017.csv', nrows=10000)

tooltips = [('Vessel ID','@vessel_id'), ('Timestamp','@t'), ('Speed (knots)','@speed'),
            ('Course over Ground (degrees)','@course'), ('Heading (degrees)','@heading'), ('Coordinates','(@lon, @lat)')]

viz_express.plot_points_on_map(st_viz, tile_provider='CARTODBPOSITRON', tools=['lasso_select'], tooltips=tooltips)

st_viz.add_numerical_filter(filter_mode='range', title='Speed (knots)', numeric_name='speed', step=1)
st_viz.show_figures(notebook=True)

Loading BokehJS ...

**Threshold-based:** Apply upper/lower bound conditions. Numerical Filter (>=)

In [ ]:
st_viz = st_visualizer(limit=1000) # Initialize a VISIONS Instance (ST Visualizer Object)
st_viz.get_data_csv(filepath=r'..\..\data\unipi_ais_dynamic_2017\unipi_ais_dynamic_dec2017.csv', nrows=10000)

tooltips = [('Vessel ID','@vessel_id'), ('Timestamp','@t'), ('Speed (knots)','@speed'),
            ('Course over Ground (degrees)','@course'), ('Heading (degrees)','@heading'), ('Coordinates','(@lon, @lat)')]

viz_express.plot_points_on_map(st_viz, tile_provider='CARTODBPOSITRON', tools=['lasso_select'], tooltips=tooltips)

st_viz.add_numerical_filter(filter_mode='>=', title='Speed (knots) >=', numeric_name='speed', step=1)
st_viz.show_figures(notebook=True)

Loading BokehJS ...

### Categorical Filter

Restrict data to specific labels or groupings. 

In [ ]:
glife = pd.read_csv('..\..\data\dummy_categorical_gdf.csv')
glife = viz_helper.create_geometry(glife, crs='epsg:4326')

st_viz = st_visualizer(limit=500)
st_viz.set_data(glife)

tooltips = [('User ID','@user_id'), ('Vehicle','@label')]
viz_express.plot_points_on_map(st_viz, tools=['lasso_select'], tooltips=tooltips)

st_viz.add_categorical_filter(title='Vehicle', categorical_name='label')
st_viz.show_figures(notebook=True, sizing_mode='stretch_both')

Loading BokehJS ...

## A Note Towards Filter Interaction

For webmercator, when latitudes tend to 90 deg, northing tend to infinity, so using 89.9999 is not the solution. 
The recommended area of use for EPSG:3857 is for latitudes between -85 and 85 degrees.
Thus, we get the locations that reside anywhere but the poles.

In [ ]:
world_airports = pd.read_csv(r'..\..\data\airports_dummy.csv')
world_airports = viz_helper.create_geometry(world_airports, coordinate_columns=['Longitude', 'Latitude'], crs=4326)

world_airports = world_airports.loc[world_airports.Latitude.between(-85, 85)].copy()
world_airports = world_airports.loc[world_airports.Altitude >= 0].copy()

In [ ]:
st_viz = st_visualizer(limit=500)
st_viz.set_data(world_airports, sp_columns=['Longitude', 'Latitude'])

tooltips = [('Name','@Name'), ('City Country','@City_Country'), ('IATA','@IATA'), ('Location','(@Longitude, @Latitude, @Altitude)'), ('Timezone', '@Timezone.1')]
viz_express.plot_points_on_map(st_viz, size=5, tools=['hover,lasso_select'], tooltips=tooltips)

st_viz.add_categorical_filter()
st_viz.add_numerical_filter(filter_mode='>=', callback_policy='value_throttled')

st_viz.figure.legend.location = "top_left"
st_viz.figure.legend.click_policy = "mute"
st_viz.figure.toolbar.active_scroll = st_viz.figure.select_one(bokeh_models.WheelZoomTool)
st_viz.show_figures(notebook=True)


Loading BokehJS ...

## Loading and Cleaning a Polygon Geometry: Saronic Gulf Example

The library is also capable of visualizing Polygon and Multipolygon Geometries. In this step, we load polygon data from OpenStreetMap to define the spatial extent of the Saronic Gulf region in Greece. However, the downloaded geometry includes multiple components (such as holes or nested shapes), which we simplify into a clean, unified polygon. The library's geometry handler uses shapely and has been migrated to 2.0



https://shapely.readthedocs.io/en/stable/migration.html

In [ ]:
url = 'http://polygons.openstreetmap.fr/get_geojson.py?id=9135811&params=0'
saronic_gdf = gpd.read_file(url)

first_polygon = list(saronic_gdf.geometry.iloc[0].geoms)[0]

geom_final = shapely.geometry.Polygon(first_polygon.exterior.coords, [inner.exterior.coords for inner in list(saronic_gdf.geometry.iloc[0].geoms)[1:]])

saronic_gulf = saronic_gdf.copy()
saronic_gulf.loc[:, saronic_gulf.geometry.name] = geom_final

### Visualizing Simple Polygons

A new VISIONS instance is initialized with allow_complex_geometries=False to ensure simpler and more performant rendering. The loaded geometry is passed to the express function that fast tracks the procedure to quickly render the polygon using the 'patches' style

In [ ]:
st_viz = st_visualizer(allow_complex_geometries=False)
st_viz.set_data(saronic_gulf)

viz_express.plot_polygons_on_map(st_viz, polygon_type='patches')
st_viz.show_figures(notebook=True, sizing_mode='stretch_both')

Loading BokehJS ...

### Visualizing Complex Geometries

In this cell, we visualize the full, complex geometry of the Saronic Gulf region — including its multiple parts (exterior and interior rings). 

In [ ]:
st_viz = st_visualizer(allow_complex_geometries=True)
st_viz.set_data(saronic_gulf)

viz_express.plot_polygons_on_map(st_viz, polygon_type='multi_polygons')
st_viz.show_figures(notebook=True, sizing_mode='stretch_both')

Loading BokehJS ...

## Bonus: Using Custom WTSTileSources

You can use your own custom tile providers for source in your visualizations. ESRI_IMAGERY Is a custom initialized `WMTSTileSource` instance and will be used to visualize the data instead of the default tile provider (CARTODBPOSITRON)

In [ ]:
ESRI_IMAGERY = bokeh_models.WMTSTileSource(
    url='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{Z}/{Y}/{X}',
    attribution=(
        "Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, "
        "Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community"
    )
)

st_viz = st_visualizer(limit=5000) # Initialize a VISIONS Instance (ST Visualizer Object)
st_viz.get_data_csv(filepath=r'..\..\data\unipi_ais_dynamic_2017\unipi_ais_dynamic_dec2017.csv', nrows=10000)

tooltips = [('Vessel ID','@vessel_id'), ('Timestamp','@t'), ('Speed (knots)','@speed'),
            ('Course over Ground (degrees)','@course'), ('Heading (degrees)','@heading'), ('Coordinates','(@lon, @lat)')]

viz_express.plot_points_on_map(st_viz, tile_provider=ESRI_IMAGERY, tools=['lasso_select'], tooltips=tooltips)

st_viz.show_figures(notebook=True, sizing_mode='stretch_both')

Loading BokehJS ...